INTAKE

Purpose: The intake is like the ”entry door” for sunlight. It’s designed to gather as much sunlight as possible and
guide it into the Quantum Photonic Array.

Basic Idea: Imagine using a magnifying glass to focus sunlight on a tiny spot; the intake works on a similar
principle. It collects a large amount of sunlight and concentrates or ”focuses” it onto a smaller area, increasing the
energy densit

In [9]:
#   SUNLIGHT INTENSITY
#   The intensity of sunlight (I0(t)) on a clear day outside the Earth’s atmosphere is approximately 1361 W/m2
#   This intensity varies based on the angle of the sun and can be modeled using the cosine of the zenith angle:

#   Where day_of_year = 1 represents January 1st.


In [3]:
latitude = 30.266666
longitude = -97.733330
# austin's cordinates

import math
import pandas as pd
from datetime import datetime, timedelta

def solar_declination_angle(day_of_year):
    # Calculate the Solar Declination Angle
    return -23.44 * math.cos(math.radians((360/365) * (day_of_year + 10)))

def solar_elevation_angle(latitude, longitude, day_of_year, hour):
    # Calculate the Solar Elevation Angle
    solar_declination = solar_declination_angle(day_of_year)
    hour_angle = 15 * (hour - 12)  # Assuming solar noon is at 12:00 PM
    return math.degrees(math.asin(
        math.sin(math.radians(latitude)) * math.sin(math.radians(solar_declination)) +
        math.cos(math.radians(latitude)) * math.cos(math.radians(solar_declination)) * math.cos(math.radians(hour_angle))
    ))

def sunlight_intensity(solar_elevation_angle):
    # Calculate Sunlight Intensity (a simple example, actual intensity depends on various factors)
    return math.sin(math.radians(solar_elevation_angle))

results = []

for day in range(0, 7, 1):  # Simulate each day
    for hour in range(0, 24, 1):  # Simulate each hour
        simulated_date = datetime(2023, 1, 1) + timedelta(days=day, hours=hour)
        day_of_year = (simulated_date - datetime(simulated_date.year, 1, 1)).days + 1

        declination_angle = solar_declination_angle(day_of_year)
        elevation_angle = solar_elevation_angle(latitude, longitude, day_of_year, hour)
        intensity = sunlight_intensity(elevation_angle)

        results.append([simulated_date, day, hour, declination_angle, elevation_angle, intensity])

df = pd.DataFrame(results, columns=["Simulated Date", "Simulated Day", "Hour of Day", "Solar Declination Angle", "Solar Elevation Angle (degrees)", "Sunlight Intensity (W/m^2)"])
df.to_csv("solar_simulation_results.csv", index=False)
